In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Apr  6 16:09:25 2018

@author: Shubadra
"""

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import numpy as np
import string
import re
from functools import reduce
from nltk.stem import PorterStemmer
from collections import Counter 
from nltk.corpus import wordnet 
from nltk.stem import WordNetLemmatizer 


# def get_pos( word ):
#     w_synsets = wordnet.synsets(word)

#     pos_counts = Counter()
#     pos_counts["n"] = len(  [ item for item in w_synsets if item.pos()=="n"]  )
#     pos_counts["v"] = len(  [ item for item in w_synsets if item.pos()=="v"]  )
#     pos_counts["a"] = len(  [ item for item in w_synsets if item.pos()=="a"]  )
#     pos_counts["r"] = len(  [ item for item in w_synsets if item.pos()=="r"]  )
    
#     most_common_pos_list = pos_counts.most_common(3)
#     return most_common_pos_list[0][0]

find = lambda searchList, elem: [[i for i, x in enumerate(searchList) if x == e] for e in elem]
fname = 'project_2_train/' + 'data 2_train.csv'

stopwords_set = set(stopwords.words('english'))
stopwords_set = {'i', 'shan', 'just', 'how', 'each', 'out', 'themselves', 'their', 'before', 'were', 'very', 'as', 'further', 'his', 'a', 'once', 'youve', 'y', 'is', 'shouldve', 'youll', 'on', 'd', 'm', 'under', 'haven', 'which', 'only', 'them', 'was', 'by', 'needn', 'whom', 'that', 'when', 's', 'isn', 'its', 'no', 'wasn', 'in', 'we', 'theirs', 'those', 'this', 'having', 'and', 'ain', 'most', 'up', 'off', 'being', 'aren', 'shouldn', 'ourselves', 'from', 'down', 'herself', 'her', 'you', 'are', 'its', 'who', 'the', 'here', 'where', 'your', 'youd', 'she', 'didn', 'weren', 'about', 'has', 'our', 'an', 'yourselves', 'or', 'hasn', 'again', 'while', 'does', 'him', 'shes', 'above', 'below', 'itself', 'to', 'through', 'will', 'couldn', 'hers', 'they', 'doing', 'because', 'he', 'what', 'such', 'youre', 'nor', 'too', 'should', 'ours', 'then', 'himself', 'all', 'of', 'mightn', 'between', 'now', 'against', 'some', 'with', 'until', 'am', 'other', 'at', 'can', 'over', 'mustn', 'wouldn', 'do', 'for', 'after', 'hadn', 'me', 'been', 'same', 'doesn', 'my', 'these', 'll', 'did', 'had', 'it', 'so', 'ma', 'during', 'than', 'o', 'yourself', 'own', 'have', 're', 've', 'be', 'why', 't', 'there', 'more', 'won', 'yours', 'few', 'into', 'thatll', 'any', 'myself', 'both', 'don', 'if'}
# print(stopwords_set)
# stopwords_set.remove('but')
# stopwords_set.remove('not')

sentence_weights = []
X, Y, X1 = [], [], []

f = open(fname, 'r')
for i, line in enumerate(f):
    if i != 0:
        ''' Splitting the columns based on comma - since it is csv'''
        columns = line.split(',')

        ''' The comma in the actual sentence was represented as [commma] because of csv format, replace that '''
        columns[1] = columns[1].replace('[comma]', ',')
        columns[2] = columns[2].replace('[comma]', ',')

        ''' Some aspect terms are mic in the sentence where computer/mic is present '''
        columns[1] = columns[1].replace('-', ' ')
        columns[2] = columns[2].replace('-', ' ')
        columns[1] = columns[1].replace('/', ' ')
        columns[2] = columns[2].replace('/', ' ')

        ''' NOt used anymore'''
        tokenizer = RegexpTokenizer(r'\w+') # doesn't work if the special char is in the token
        # columns[2] = tokenizer.tokenize(columns[2].lower())

        '''Tokenize the words'''
        columns[1] = word_tokenize(columns[1])
        columns[2] = word_tokenize(columns[2])

        for j, elem in enumerate(columns[1]):
            '''Remove special characters'''
            columns[1][j] = re.sub('[^0-9a-zA-Z]+', '', elem).lower()
            ''' didn't is tokenized as did n't hence following code '''
            if columns[1][j] == 'nt':
                columns[1][j] = 'not'

        for j, elem in enumerate(columns[2]):
            '''Remove special characters'''
            columns[2][j] = re.sub('[^0-9a-zA-Z]+', '', elem).lower()
            ''' didn't is tokenized as did n't hence following code '''
            if columns[2][j] == 'nt':
                columns[2][j] = 'not'

        '''Remove empty string tokens'''
        columns[1] = [x.strip() for x in columns[1] if x.strip() != '']
        columns[2] = [x.strip() for x in columns[2] if x.strip() != '']

        ''' Remove stop words '''
        columns[1] = [word for word in columns[1] if word not in stopwords_set]
        columns[1] = [word for word in columns[1] if word not in stopwords_set]
        columns[2] = [word for word in columns[2] if word not in stopwords_set]
        
#         '''Lemmatizing followed by Stemming'''
#         ps = PorterStemmer()
#         wnl = WordNetLemmatizer()
#         for word_index, word in enumerate(columns[1]):
#             columns[1][word_index] = ps.stem(word)
#         for aspect_term_index, aspect_term in enumerate(columns[2]):
#             columns[1][aspect_term_index] = ps.stem(wnl.lemmatize(aspect_term))
 
        columns[1] = ' '.join(columns[1])
        columns[2] = ' '.join(columns[2])

        ''' The aspect term location given is not proper - hence extracting the location by ourselves '''
        columns[3] = []
        ''' Finds the location of the aspect term in the sentence '''
        for m in re.finditer(columns[2], columns[1]):
            columns[3].append([m.start(), m.end()])
            ''' since some aspect terms are not surrounded by spaces, hence below code
            NOT USED ANYMORE
            '''
            if m.start() != 0 and columns[1][m.start() - 1] != ' ':
                # columns[1] = columns[1][:m.start()] + ' ' + columns[1][m.start():]
                pass

        ''' Because of adding space in above code, the position is messed up, hence redoing it. '''
        columns[3] = []

        for m in re.finditer(columns[2], columns[1]):
            columns[3].append([m.start(), m.end()])

        ''' Tokenizing the words again '''
        columns[1] = columns[1].split(' ')
        columns[2] = columns[2].split(' ')
        ''' New column to specify the aspect term location in the list '''
        columns.append([])
        # columns[5] = []
        ''' for multiple positions of aspect term '''
        for elem in columns[3]:
            if elem[0] == 0: #start index of the 1st occurance of the aspect term
                columns[5].append([0])
            else:
                temp_len = elem[0] # assign the start position of the aspect term
                for j, tokens in enumerate(columns[1]):
                    temp_len -= (len(tokens) + 1) # Counting the words till the position - +1 for blank space
                    if temp_len == 0: # Reached the aspect word
                        columns[5].append([j+1])
                        break
            for k in range(len(columns[2])-1): # if multiple words in the aspect term - tag the following words in the sentence
                columns[5][-1].append(columns[5][-1][-1]+1)

        ''' if not found the aspect term location in the list then do '''
        if len(columns[5]) < 1:
            continue

            # pass

        word_wt_list = [[elem, 0] for elem in columns[1]]
        ''' Assigning weights to every word based on the distance from the aspect term '''
        for j, elem in enumerate(columns[5]):
            for k, word in enumerate(columns[1]):
                if k < elem[0]: # Word left to the aspect term
                    dist = abs(elem[0] - k)
                    if word_wt_list[k][1] < 1/dist:
                        word_wt_list[k][1] = 1/dist
                elif k > elem[-1]: # word right to the aspect term : Aspect term can have multiple words
                    dist = abs(k - elem[-1])
                    if word_wt_list[k][1] < 1/dist:
                        word_wt_list[k][1] = 1/dist
            for aspect_word_loc in elem:
                word_wt_list[aspect_word_loc][1] = 1.5

        ''' For duplicate words - if it's aspect term - add the weights, if not, then take the weight which is greater, make the other weight as zero '''
        for j, word_1 in enumerate(word_wt_list):
            for k, word_2 in enumerate(word_wt_list[j+1:]):
                if word_1[0] == word_2[0]:
                    if word_1[1] == 1.5 and word_2[1] == 1.5:
                        # print(line)
                        # print(word_wt_list)
                        word_1[1] += 1.5
                        word_2[1] = 0
                    elif word_1[1] < word_2[1]:
                        # print(line)
                        # print(word_wt_list)
                        word_1[1] = word_2[1]
                        word_2[1] = 0
                    elif word_1[1] > word_2[1]:
                        # print(line)
                        # print(word_wt_list)
                        word_2[1] = 0
                    elif word_1[1] == word_2[1]:
                        word_2[1] = 0


        ''' For removing duplicate words - they have weight zero now'''
        word_wt_list = [elem for elem in word_wt_list if elem[1] != 0]

        if len(columns[2]) > 1 and len(columns[5]) > 1:
            # print(line)
            # print(columns)
            # print(word_wt_list)
            pass

        if columns[0] == '2911_0':
            # print(line)
            # print(word_wt_list)
            pass
        if i == 100:
            # break
            pass

        '''Removing new line character from columns[4] -  the polarity column'''
        columns[4] = columns[4].rstrip('\n')
        X.append(columns[1])
        X1.append(' '.join(columns[1]))

        Y.append(columns[4])
        sentence_weights.append(word_wt_list)


'''Building vocabulary and counting word occurances'''
X_reduced = reduce(lambda x1, x2: x1 + x2, X)
vocab = list(set(X_reduced))
print(vocab[0])
weight_v = np.zeros_like(vocab, dtype = np.float_)
weight_x = []
pf = open('weightxy_data2.csv', 'w')

pf.write(','.join(vocab))
pf.write(',class_')
pf.write('\n')
for i, sentence in enumerate(sentence_weights):
    for word in sentence:
        v_index = vocab.index(word[0])
        weight_v[v_index] = word[1]

    weight_x.append(weight_v)
    if len(sentence) != weight_v[np.where(weight_v > 0)].shape[0]:
        print(sentence)
        print(len(sentence))
        print(weight_v[np.where(weight_v > 0)])

    ''' code to write the data to file '''
    temp = [str(j) for j in weight_v.tolist()]
    pf.write(','.join(temp))
    pf.write(','+str(Y[i]))
    pf.write('\n')
    weight_v = np.zeros_like(weight_v)

pf.close()
weight_x = np.array(weight_x)

print(weight_x.shape)

f.close()



# weight_X2 = np.reshape(weight_X, (len(Y) ,weight_X.shape[0]))
# print(weight_X2.shape)
# tokens = f.split(',') # columns
# remove punctuations
# table = str.maketrans('', '', string.punctuation)
# print(table)
# tokens = [w.translate(table) for w in tokens]
# # remove tokens that are not alphbetic
# tokens = [word for word in tokens if word.isalpha()]
#
# # remove stop words
# stop_words = set(stopwords.words('english'))
# tokens = [w for w in tokens if not w in stop_words]
# # filter out short tokens
# tokens = [word for word in tokens if len(word) > 1]
# print(tokens)


sense
(3602, 3612)


In [2]:
import pandas as pd
from sklearn import svm

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, classification_report
# from sklearn.metrics import accuracy_score
import pickle
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
print('success')
'hello world'

success


'hello world'

In [3]:
'''Load data set'''
df1 = pd.read_csv("weightxy_data2.csv")
print(df1.shape)
print(df1.iloc[:,:-1].shape)
'hello world'

(3602, 3613)
(3602, 3612)


'hello world'

In [4]:
# X_train, X_test, Y_train, Y_test = train_test_split(df1.iloc[:,:-1], df1['class_'], test_size = 0.3, random_state =42)
X, Y = df1.iloc[:,:-1], df1['class_']

# svm_clf = svm.SVC(kernel = 'rbf')
svm_clf = svm.LinearSVC(C = 0.1, random_state = 0) #loss = 'hinge') #, multi_class = 'crammer_singer')

# svm_clf.fit(X, Y)
print('success')
print(X.shape)
print(Y.shape)
'hello world'

success
(3602, 3612)
(3602,)


'hello world'

In [5]:
y_pred = cross_validate(svm_clf, X, Y, cv = 10, scoring = 'accuracy')

print(y_pred)
print('success')
'hello world'


{'fit_time': array([ 0.22421408,  0.23439264,  0.21877432,  0.21277785,  0.34834719,
        0.21967864,  0.21164298,  0.20967221,  0.21877408,  0.22537494]), 'score_time': array([ 0.22360039,  0.        ,  0.        ,  0.01562786,  0.01404166,
        0.01562929,  0.        ,  0.        ,  0.        ,  0.        ]), 'test_score': array([ 0.72099448,  0.72651934,  0.73480663,  0.66204986,  0.72222222,
        0.70194986,  0.70194986,  0.67688022,  0.70473538,  0.67966574]), 'train_score': array([ 0.90432099,  0.90617284,  0.90401235,  0.91237272,  0.90746453,
        0.90595128,  0.90718471,  0.91026827,  0.90595128,  0.90595128])}
success


'hello world'

In [6]:
'''Cross validation'''
y_pred = cross_val_predict(svm_clf, X, Y, cv = 10) #, scoring = 'accuracy')

print(y_pred)
print('success')
'hello world'


mod_accracy = accuracy_score(y_pred, Y)
print('\n\nModel Accuracy is: {:.4%}' .format(mod_accracy))

target_names = ['0', '1', '-1']
print('Pr, Re scores wrt each class')
print(classification_report(y_pred, Y, target_names=target_names))


[-1 -1  1 ...,  1  0  0]
success


Model Accuracy is: 70.3220%
Pr, Re scores wrt each class
             precision    recall  f1-score   support

          0       0.49      0.60      0.54       653
          1       0.31      0.52      0.39       384
         -1       0.90      0.76      0.82      2565

avg / total       0.76      0.70      0.72      3602



In [ ]:
print(scores)

In [9]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaBoost_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2)) #, n_estimators=2, learning_rate=1)
scores = cross_validate(adaBoost_clf, X, Y, cv = 10)
print(scores.keys())
print('success')
'hello world'

print(scores)

y_pred = cross_val_predict(adaBoost_clf, X, Y, cv = 10) #, scoring = 'accuracy')

print(y_pred)
print('success')
'hello world'


mod_accracy = accuracy_score(y_pred, Y)
print('\n\nModel Accuracy is: {:.4%}' .format(mod_accracy))

target_names = ['0', '1', '-1']
print('Pr, Re scores wrt each class')
print(classification_report(y_pred, Y, target_names=target_names))


dict_keys(['fit_time', 'score_time', 'test_score', 'train_score'])
success
{'fit_time': array([ 5.25342131,  5.08987379,  5.27256608,  6.38939095,  5.57181549,
        5.61872816,  5.44238305,  5.18801045,  5.29704666,  5.19146609]), 'score_time': array([ 0.03128791,  0.04699802,  0.04685092,  0.04311419,  0.04311299,
        0.04683065,  0.05353093,  0.04688287,  0.03130293,  0.02844381]), 'test_score': array([ 0.67955801,  0.63535912,  0.64917127,  0.61218837,  0.68611111,
        0.64066852,  0.63509749,  0.64623955,  0.61002786,  0.66852368]), 'train_score': array([ 0.77438272,  0.77561728,  0.76851852,  0.77198396,  0.76588526,
        0.77489978,  0.77181622,  0.7613321 ,  0.76441566,  0.76688252])}
[-1 -1  1 ..., -1 -1 -1]
success


Model Accuracy is: 64.8529%
Pr, Re scores wrt each class
             precision    recall  f1-score   support

          0       0.38      0.48      0.42       632
          1       0.28      0.45      0.34       387
         -1       0.86      0.72 

In [7]:
# Using MNB: 
from sklearn.naive_bayes import MultinomialNB
mnb_clf = MultinomialNB()

scores = cross_validate(mnb_clf, X, Y, cv = 10)
print(scores.keys())
print('success')
'hello world'

print(scores)

y_pred = cross_val_predict(mnb_clf, X, Y, cv = 10) #, scoring = 'accuracy')

print(y_pred)
print('success')
'hello world'


mod_accracy = accuracy_score(y_pred, Y)
print('\n\nModel Accuracy is: {:.4%}' .format(mod_accracy))

target_names = ['0', '1', '-1']
print('Pr, Re scores wrt each class')
print(classification_report(y_pred, Y, target_names=target_names))



dict_keys(['fit_time', 'score_time', 'test_score', 'train_score'])
success
{'fit_time': array([ 0.19217801,  0.0937593 ,  0.10939217,  0.10822964,  0.09376216,
        0.10864186,  0.12501478,  0.11818719,  0.12501001,  0.11589599]), 'score_time': array([ 0.00501418,  0.01563025,  0.        ,  0.01562786,  0.01562476,
        0.0156281 ,  0.        ,  0.        ,  0.        ,  0.        ]), 'test_score': array([ 0.70165746,  0.69060773,  0.66022099,  0.65373961,  0.67222222,
        0.64345404,  0.65738162,  0.61002786,  0.66573816,  0.62674095]), 'train_score': array([ 0.84259259,  0.84753086,  0.84598765,  0.84757791,  0.84577421,
        0.84736355,  0.8470552 ,  0.84828862,  0.84643848,  0.84304656])}
[-1 -1  1 ...,  1  1  1]
success


Model Accuracy is: 65.8245%
Pr, Re scores wrt each class
             precision    recall  f1-score   support

          0       0.44      0.55      0.49       638
          1       0.29      0.40      0.33       452
         -1       0.85      0.73 

In [8]:
from sklearn.ensemble import RandomForestClassifier
randomForest_clf = RandomForestClassifier(max_depth=2, random_state=4)

scores = cross_validate(randomForest_clf, X, Y, cv = 10)
print(scores.keys())
print('success')
'hello world'

print(scores)

y_pred = cross_val_predict(randomForest_clf, X, Y, cv = 10) #, scoring = 'accuracy')

print(y_pred)
print('success')
'hello world'


mod_accracy = accuracy_score(y_pred, Y)
print('\n\nModel Accuracy is: {:.4%}' .format(mod_accracy))

target_names = ['0', '1', '-1']
print('Pr, Re scores wrt each class')
print(classification_report(y_pred, Y, target_names=target_names))


dict_keys(['fit_time', 'score_time', 'test_score', 'train_score'])
success
{'fit_time': array([ 0.1562264 ,  0.10944152,  0.10939145,  0.09377956,  0.10448217,
        0.10942316,  0.10942674,  0.1093874 ,  0.11043811,  0.10944247]), 'score_time': array([ 0.        ,  0.01557374,  0.        ,  0.01558328,  0.01559043,
        0.        ,  0.01558948,  0.        ,  0.00601602,  0.01557326]), 'test_score': array([ 0.59944751,  0.59944751,  0.59944751,  0.60110803,  0.6       ,
        0.60167131,  0.60167131,  0.60167131,  0.60167131,  0.60167131]), 'train_score': array([ 0.60092593,  0.60216049,  0.60092593,  0.60074051,  0.60086366,
        0.60160345,  0.60098674,  0.60067838,  0.60067838,  0.60160345])}
[1 1 1 ..., 1 1 1]
success


Model Accuracy is: 60.0777%
Pr, Re scores wrt each class
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         0
          1       0.00      0.00      0.00         0
         -1       1.00      0.60      0

C:\Users\Shubadra\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
# '''calculate the precision recall and stuff'''

# t_precision = precision_score(Y_test, t_pred, average='weighted')
# t_recall = recall_score(Y_test, t_pred, average='weighted')
# t_f1_score = f1_score(Y_test, t_pred, average='weighted')

# print('*** Scores for SVM: ***')
# target_names = ['0', '1', '-1']
# print('Pr, Re scores wrt each class')
# print(classification_report(Y_test, t_pred, target_names=target_names))

# print('\n\nModel Accuracy is: {:.4%}' .format(mod_accracy))
# print('\n\nPrecision Score is: {:.4%}' .format(t_precision))
# print('\n\nRecall Score is: {:.4%}' .format(t_recall))
# print('\n\nF1-Score is: {:.4%}' .format(t_f1_score))
